In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

import nltk
from sklearn.decomposition import TruncatedSVD, PCA
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from collections import Counter

In [2]:
titles = []
with open(os.path.join('data', 'all_book_titles.txt'), encoding = 'utf-8') as f :
    for line in f :
        line = line.strip().lower()
        if line:
            titles.append(line)
            
stopwords = set()
with open(os.path.join('data', 'stopwords.txt'), encoding = 'utf-8') as f :
    for line in f :
        line = line.strip().lower()
        if line:
            stopwords.add(line)

stopwords.union({'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 
                 'ourselves', 'you', "you're", "you've", "you'll", 
                 "you'd", 'your', 'yours', 'yourself', 'yourselves', 
                 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 
                 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 
                 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 
                 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 
                 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
                 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 
                 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                 'against', 'between', 'into', 'through', 'during', 'before', 
                 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
                 'out', 'on', 'off', 'over', 'under', 'again', 'further', 
                 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 
                 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 
                 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
                 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 
                 "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 
                 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', 
                 "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
                 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 
                 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
                 "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 
                 'wouldn', "wouldn't", "edition", "series", "introduction", "application"
                "first", 'second', 'third', 'fourth', 'guide', 'brief'})
            
print(f'{len(titles)} titles found.\n{len(stopwords)} stopwords found.')

2373 titles found.
420 stopwords found.


In [35]:
def my_tokenize_function(text):
    token_list = nltk.tokenize.word_tokenize(text)
    token_list = [word_net_lemmatizer.lemmatize(token) for token in token_list 
                  if len(token) > 2 
                  and token.isalpha() 
                  and token not in stopwords]
    return token_list

word_net_lemmatizer = WordNetLemmatizer()
tokenizer = Tokenizer(analyzer=my_tokenize_function)
tokenizer.fit_on_texts(titles)
print(f'Unique tokens : {len(tokenizer.word_index)}')

Unique tokens : 1978


In [42]:
sequences = tokenizer.texts_to_sequences(titles)

X = []
for sequence in sequences :
    counter = Counter(tokenizer.index_word.keys())
    counter.update(sequence)
    counter.subtract(Counter(tokenizer.index_word.keys()))
    x = list(counter.values())
    X.append(x)
    
X = np.array(X) #shape = (N, D) Document Term matrix
X_t = X.T
print(X_t, X_t.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]] (1978, 2373)


In [50]:
svd = TruncatedSVD()
Z = svd.fit_transform(X)
Z.shape

(2373, 2)

In [51]:
%matplotlib qt
plt.scatter(Z[:, 0], Z[:, 1])
for i in range(1, len(tokenizer.word_index)) :
    plt.annotate(text = tokenizer.index_word[i], xy = (Z[i, 0], Z[i, 1]))

In [48]:
plt.boxplot(Z);